In [ ]:
# read input
f=open('dataset/sentiment.txt','r')
lines=f.readlines()
f.close()
print (len(lines))

In [2]:
#helper functions for data cleaning
import re
import numpy as np

stopwords = ["a", "about", "above", "above", "across", "after", "afterwards", "again", "against", "all", "almost", "alone", "along", "already", "also","although","always","am","among", "amongst", "amoungst", "amount",  "an", "and", "another", "any","anyhow","anyone","anything","anyway", "anywhere", "are", "around", "as",  "at", "back","be","became", "because","become","becomes", "becoming", "been", "before", "beforehand", "behind", "being", "below", "beside", "besides", "between", "beyond", "bill", "both", "bottom","but", "by", "call", "can", "cannot", "cant", "co", "con", "could", "couldnt", "cry", "de", "describe", "detail", "do", "done", "down", "due", "during", "each", "eg", "eight", "either", "eleven","else", "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone", "everything", "everywhere", "except", "few", "fifteen", "fify", "fill", "find", "fire", "first", "five", "for", "former", "formerly", "forty", "found", "four", "from", "front", "full", "further", "get", "give", "go", "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", "however", "hundred", "ie", "if", "in", "inc", "indeed", "interest", "into", "is", "it", "its", "itself", "keep", "last", "latter", "latterly", "least", "less", "ltd", "made", "many", "may", "me", "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly", "move", "much", "must", "my", "myself", "name", "namely", "neither", "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on", "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", "out", "over", "own","part", "per", "perhaps", "please", "put", "rather", "re", "same", "see", "seem", "seemed", "seeming", "seems", "serious", "several", "she", "should", "show", "side", "since", "sincere", "six", "sixty", "so", "some", "somehow", "someone", "something", "sometime", "sometimes", "somewhere", "still", "such", "system", "take", "ten", "than", "that", "the", "their", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they", "thickv", "thin", "third", "this", "those", "though", "three", "through", "throughout", "thru", "thus", "to", "together", "too", "top", "toward", "towards", "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us", "very", "via", "was", "we", "well", "were", "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who", "whoever", "whole", "whom", "whose", "why", "will", "with", "within", "without", "would", "yet", "you", "your", "yours", "yourself", "yourselves", "the"]

def text_cleaner(text): 
    text=remove_link(text.lower())
    long_words=[]
    for i in text.split():
        if i not in stopwords:                  
            long_words.append(i)
    return long_words

def remove_link(text):
    regex = r'https?://[^\s<>)"‘’]+'
    match = re.sub(regex,' ', text)
    regex = r'https?:|urls?|[/\:,-."\'?!;…]+'
    tweet = re.sub(regex,' ', match)
    tweet = re.sub("[^a-zA-Z_]", " ", tweet)
    tweet = re.sub("[ ]+", " ", tweet) 
    return tweet


In [3]:
#divide the data into train set and test set
import random
random.seed(1)
random.shuffle(lines) #shuffle the dataset before dividing it into train and test set
split_size = int(0.8*len(lines)) #use 80% of total data as train set and 20% as test set
train_lines = lines[:split_size]
test_lines = lines[split_size:]

print ("Training set size : ", len(train_lines))
print ("Test set size : ", len(test_lines))

Training set size :  914
Test set size :  229


In [12]:
#convert string tokens to integers
#create a vocabulary set and assign a unique id to each word in the vocabulary

#load all unique vocabulary
vocab = []
maxlen = []
for l in train_lines:
    s = l.strip().split('\t')
    words = text_cleaner(s[0].strip())
    vocab += words
    maxlen.append(len(words))
vocab = list(set(vocab))
print ("Vocabulary size : ", len(vocab))

#assign unique id to each vocabulary
word2id = dict()
id2word = dict()
for i,v in enumerate(vocab,1):
    word2id[v] = i
    id2word[i] = v 
word2id['PAD'] = 0 #special token to take care of unseen words in the test set
id2word[0] = 'PAD'
maxlen = max(maxlen)
print ("Max sentence length : ",maxlen)

Vocabulary size :  4012
Max sentence length :  19


In [5]:
#Prepare train and test set
#Convert strings to integers
#prepare train and test set

#prepare train set
import numpy as np
train_X = []
train_Y = []
for l in train_lines:
    s = l.strip().split('\t')
    text = text_cleaner(s[0])
    label = int(s[1].strip())
    temp_x = [word2id[x] for x in text]
    temp_x += [0]* (maxlen-len(temp_x)) #convert all input to equal size to enable training in batches
    temp_y = [0]*2
    temp_y[label] = 1
    train_X.append(temp_x)
    train_Y.append(temp_y)
print (len(train_X))
print (len(train_Y))
print (train_X[0])
print (train_Y[:5])
train_X = np.array(train_X)
train_Y = np.array(train_Y) 

914
914
[2044, 1977, 873, 917, 1386, 3161, 3161, 705, 3734, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[1, 0], [1, 0], [1, 0], [1, 0], [1, 0]]


In [6]:
test_X = []
test_Y = []

for l in test_lines:
    s = l.strip().split('\t')
    text = text_cleaner(s[0].strip())
    label = int(s[1].strip())
    temp_x = [word2id[x] if x in word2id else 0 for x in text ][:maxlen]
    temp_x += [0]* (maxlen-len(temp_x)) #convert all input to equal size to enable training in batches
    temp_y = [0]*2
    temp_y[label] = 1
    test_X.append(temp_x)
    test_Y.append(temp_y)
print (len(test_X))
print (len(test_Y))
print (test_X[0])
print (test_Y[0])
test_X = np.array(test_X)
test_Y = np.array(test_Y)

229
229
[2458, 923, 2888, 1081, 0, 0, 1422, 3931, 1652, 3734, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0]


In [7]:
from keras.models import Model
from keras.layers import Input, Dense, Dropout, Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import LSTM, Bidirectional,SimpleRNN

In [8]:
#sentiment classification using CNN
embedding_size = 50
input=Input(shape=(maxlen,))
embedding = Embedding(len(word2id), embedding_size, input_length=maxlen, trainable=True)(input)
convolution = Conv1D(128,3,padding='same',activation='relu',strides=1)(embedding)
pooling = GlobalMaxPooling1D()(convolution)
output = Dense(2,activation='softmax')(pooling)
model=Model(input,output)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_X, train_Y,batch_size=512,epochs=10)



Epoch 1/10
2/2 [==============================] - 0s 18ms/step - loss: 0.6816 - accuracy: 0.6871
Epoch 2/10
2/2 [==============================] - 0s 17ms/step - loss: 0.6560 - accuracy: 0.6904
Epoch 3/10
2/2 [==============================] - 0s 18ms/step - loss: 0.6344 - accuracy: 0.6904
Epoch 4/10
2/2 [==============================] - 0s 17ms/step - loss: 0.6176 - accuracy: 0.6904
Epoch 5/10
2/2 [==============================] - 0s 17ms/step - loss: 0.6017 - accuracy: 0.6904
Epoch 6/10
2/2 [==============================] - 0s 17ms/step - loss: 0.5888 - accuracy: 0.6904
Epoch 7/10
2/2 [==============================] - 0s 17ms/step - loss: 0.5767 - accuracy: 0.6904
Epoch 8/10
2/2 [==============================] - 0s 17ms/step - loss: 0.5670 - accuracy: 0.6904
Epoch 9/10
2/2 [==============================] - 0s 18ms/step - loss: 0.5570 - accuracy: 0.6904
Epoch 10/10
2/2 [==============================] - 0s 18ms/step - loss: 0.5467 - accuracy: 0.6904


In [9]:
res = model.evaluate(test_X,test_Y)


8/8 [==============================] - 0s 2ms/step - loss: 0.6399 - accuracy: 0.6463


In [10]:
p = model.predict(test_X)
for i,l in enumerate(test_lines[:5]):
  print (l.strip(),p[i])

We blame cities for the majority of CO2 emissions without acknowledging their vulnerability to #CFCC15 #journey2015 #S2228 #SemST	0 [0.7244334  0.27556658]
Feminists who go for a gender studies degree should also blame the patriarchy for their mediocre grades in science. #SemST	0 [0.72194064 0.2780594 ]
Just wrote my blog to help @CalAlimony pass a vital law that ends #alimony. Posting soon. #Divorce #leanin #SemST	1 [0.70858717 0.2914128 ]
RT @JohnFugelsang: They should just make the GOP primaries a reality game show called "Who Wants To Get Beat Up By A Girl? #SemST	0 [0.71470004 0.28529996]
It's incredibly easy to identify shitty females with a poor view on the world and what's important thanks to #SemST	0 [0.72255796 0.27744207]


In [11]:
f = open('embeddings/words.txt')
words = f.readlines()
f.close()
f = open('embeddings/vectors.txt')
vectors = f.readlines()
f.close()

embeddings = dict()
for i,w in enumerate(words):
  embeddings[w.strip()] = np.array([float(x) for x in vectors[i].strip().split('\t')])
embedding_size = len(vectors[i].strip().split('\t'))


In [13]:
embeddings_matrix = []
for i in range(len(word2id)):
  if id2word[i] in embeddings:
    embeddings_matrix.append(embeddings[id2word[i]])
  else:
    embeddings_matrix.append(np.array([0.0]*embedding_size))
embeddings_matrix = np.array(embeddings_matrix)

In [14]:
#Sentiment classification using CNN and pre-trained embeddings
embedding_size = 50
input=Input(shape=(maxlen,))
embedding = Embedding(len(word2id), embedding_size, input_length=maxlen, weights = [embeddings_matrix],trainable=True)(input)
convolution = Conv1D(128,3,padding='same',activation='relu',strides=1)(embedding)
pooling = GlobalMaxPooling1D()(convolution)
output = Dense(2,activation='softmax')(pooling)
model=Model(input,output)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_X, train_Y,batch_size=512,epochs=10)

Epoch 1/10
2/2 [==============================] - 0s 18ms/step - loss: 1.1162 - accuracy: 0.3829
Epoch 2/10
2/2 [==============================] - 0s 17ms/step - loss: 0.5973 - accuracy: 0.6991
Epoch 3/10
2/2 [==============================] - 0s 18ms/step - loss: 0.6641 - accuracy: 0.6980
Epoch 4/10
2/2 [==============================] - 0s 18ms/step - loss: 0.6872 - accuracy: 0.7112
Epoch 5/10
2/2 [==============================] - 0s 18ms/step - loss: 0.6183 - accuracy: 0.7374
Epoch 6/10
2/2 [==============================] - 0s 19ms/step - loss: 0.5373 - accuracy: 0.7593
Epoch 7/10
2/2 [==============================] - 0s 18ms/step - loss: 0.5071 - accuracy: 0.7659
Epoch 8/10
2/2 [==============================] - 0s 18ms/step - loss: 0.5225 - accuracy: 0.7549
Epoch 9/10
2/2 [==============================] - 0s 19ms/step - loss: 0.5294 - accuracy: 0.7451
Epoch 10/10
2/2 [==============================] - 0s 18ms/step - loss: 0.5017 - accuracy: 0.7702


In [15]:
res = model.evaluate(test_X,test_Y)

8/8 [==============================] - 0s 2ms/step - loss: 0.5768 - accuracy: 0.6900


In [17]:
#sentiment classification using vanilla RNN
embedding_size = 50
input=Input(shape=(maxlen,))
embedding = Embedding(len(word2id), embedding_size, input_length=maxlen, weights = [embeddings_matrix],trainable=True)(input)
rnn = SimpleRNN(64,activation='tanh')(embedding)
output = Dense(2,activation='softmax')(rnn)
model=Model(input,output)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_X, train_Y,batch_size=512,epochs=10)

Epoch 1/10
2/2 [==============================] - 0s 15ms/step - loss: 0.7079 - accuracy: 0.5263
Epoch 2/10
2/2 [==============================] - 0s 17ms/step - loss: 0.6241 - accuracy: 0.6674
Epoch 3/10
2/2 [==============================] - 0s 13ms/step - loss: 0.5933 - accuracy: 0.6947
Epoch 4/10
2/2 [==============================] - 0s 13ms/step - loss: 0.5809 - accuracy: 0.6947
Epoch 5/10
2/2 [==============================] - 0s 13ms/step - loss: 0.5613 - accuracy: 0.7177
Epoch 6/10
2/2 [==============================] - 0s 14ms/step - loss: 0.5371 - accuracy: 0.7385
Epoch 7/10
2/2 [==============================] - 0s 15ms/step - loss: 0.5088 - accuracy: 0.7593
Epoch 8/10
2/2 [==============================] - 0s 13ms/step - loss: 0.4800 - accuracy: 0.7888
Epoch 9/10
2/2 [==============================] - 0s 13ms/step - loss: 0.4605 - accuracy: 0.8096
Epoch 10/10
2/2 [==============================] - 0s 14ms/step - loss: 0.4431 - accuracy: 0.8173


In [18]:
res = model.evaluate(test_X,test_Y)

8/8 [==============================] - 0s 2ms/step - loss: 0.5931 - accuracy: 0.7249


In [20]:
#sentiment classification using LSTM
embedding_size = 50
input=Input(shape=(maxlen,))
embedding = Embedding(len(word2id), embedding_size, input_length=maxlen, weights = [embeddings_matrix],trainable=True)(input)
rnn = LSTM(64,activation='tanh')(embedding)
output = Dense(2,activation='softmax')(rnn)
model=Model(input,output)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_X, train_Y,batch_size=512,epochs=10)

Epoch 1/10
2/2 [==============================] - 0s 43ms/step - loss: 0.6807 - accuracy: 0.6368
Epoch 2/10
2/2 [==============================] - 0s 36ms/step - loss: 0.6430 - accuracy: 0.6904
Epoch 3/10
2/2 [==============================] - 0s 36ms/step - loss: 0.6181 - accuracy: 0.6904
Epoch 4/10
2/2 [==============================] - 0s 36ms/step - loss: 0.5987 - accuracy: 0.6904
Epoch 5/10
2/2 [==============================] - 0s 36ms/step - loss: 0.5902 - accuracy: 0.6904
Epoch 6/10
2/2 [==============================] - 0s 35ms/step - loss: 0.5848 - accuracy: 0.6904
Epoch 7/10
2/2 [==============================] - 0s 40ms/step - loss: 0.5746 - accuracy: 0.6904
Epoch 8/10
2/2 [==============================] - 0s 37ms/step - loss: 0.5597 - accuracy: 0.6915
Epoch 9/10
2/2 [==============================] - 0s 36ms/step - loss: 0.5471 - accuracy: 0.7101
Epoch 10/10
2/2 [==============================] - 0s 36ms/step - loss: 0.5347 - accuracy: 0.7407


In [21]:
res = model.evaluate(test_X,test_Y)

8/8 [==============================] - 0s 3ms/step - loss: 0.5707 - accuracy: 0.7118


In [22]:
#sentiment classification using Bidirectional LSTM
embedding_size = 50
input=Input(shape=(maxlen,))
embedding = Embedding(len(word2id), embedding_size, input_length=maxlen, weights = [embeddings_matrix],trainable=True)(input)
rnn = Bidirectional(LSTM(64,activation='tanh'))(embedding)
output = Dense(2,activation='softmax')(rnn)
model=Model(input,output)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_X, train_Y,batch_size=512,epochs=10)

Epoch 1/10
2/2 [==============================] - 0s 74ms/step - loss: 0.6941 - accuracy: 0.5011
Epoch 2/10
2/2 [==============================] - 0s 65ms/step - loss: 0.6140 - accuracy: 0.6893
Epoch 3/10
2/2 [==============================] - 0s 65ms/step - loss: 0.5849 - accuracy: 0.6904
Epoch 4/10
2/2 [==============================] - 0s 64ms/step - loss: 0.5746 - accuracy: 0.6926
Epoch 5/10
2/2 [==============================] - 0s 65ms/step - loss: 0.5561 - accuracy: 0.7144
Epoch 6/10
2/2 [==============================] - 0s 65ms/step - loss: 0.5350 - accuracy: 0.7363
Epoch 7/10
2/2 [==============================] - 0s 64ms/step - loss: 0.5182 - accuracy: 0.7582
Epoch 8/10
2/2 [==============================] - 0s 70ms/step - loss: 0.5083 - accuracy: 0.7713
Epoch 9/10
2/2 [==============================] - 0s 67ms/step - loss: 0.4969 - accuracy: 0.7823
Epoch 10/10
2/2 [==============================] - 0s 70ms/step - loss: 0.4836 - accuracy: 0.7856


In [23]:
res = model.evaluate(test_X,test_Y)

8/8 [==============================] - 0s 4ms/step - loss: 0.5396 - accuracy: 0.7205


In [24]:
#sequence tagginging example
fp=open('dataset/seq_tagging_dataset.txt')
lines = fp.readlines()
fp.close()
print (len(lines))

100


In [25]:
#split data into train and test set
split_size = int(0.8*len(lines)) 
train_lines = lines[:split_size]
test_lines = lines[split_size:]
print (len(test_lines))
print (len(train_lines))

20
80


In [26]:
#assign unique ids to words and labels

#first load all vocabulary and labels
vocab = []
classes = []
maxlen = []
for l in train_lines:
  s = l.strip().split()
  words = [w.strip().split('/')[0] for w in s]
  labels = [w.strip().split('/')[1] for w in s]
  vocab += words
  classes += labels
  maxlen.append(len(words))
vocab = list(set(vocab))
classes = list(set(classes))
maxlen = len(maxlen)
print ("Vocabulary size : ", len(vocab))
print ("Number of classes : ", len(classes))
print ("Max sentence length : ", maxlen)
maxlen = 10
print ("Set max length to 10", maxlen)

Vocabulary size :  1032
Number of classes :  6
Max sentence length :  80
Set max length to 10 10


In [27]:
#assin unique id to each word
word2id = dict()
id2word = dict()
for i,v in enumerate(vocab,1):
  word2id[v] = i
  id2word[i] = v
word2id['PAD'] = 0
id2word[0] = 'PAD'

In [28]:
print ("Dictionary size : ", len(word2id))

Dictionary size :  1033


In [30]:
#assign unique id to each class
class2id = dict()
id2class = dict()
for i,c in enumerate(classes):
  class2id[c] = i
  id2class[i] = c
print ("Class dictionary size : ", len(class2id))

Class dictionary size :  6


In [31]:
#prepare training and test data
#input : list of tokens
#output : lits of labels corresponding to each token

#training data preparation
train_X = []
train_Y = []
for l in train_lines:
  s = l.strip().split()
  words = [w.strip().split('/')[0] for w in s]
  labels = [w.strip().split('/')[1] for w in s]
  words = words[:maxlen]
  labels = labels[:maxlen]
  temp_x = []
  temp_y = []
  temp_x = [word2id[w] if w in word2id else 0 for w in words]
  temp_y = []
  for label in labels:
    y = [0]*len(class2id)
    y[class2id[label]] = 1
    temp_y.append(y)
  padlen = maxlen - len(words)
  temp_x += [0]*padlen
  temp_y += ([[0]*len(class2id)])*padlen
  train_X.append(temp_x)
  train_Y.append(temp_y)
print (len(train_X),len(train_Y))
train_X = np.array(train_X)
train_Y = np.array(train_Y)


80 80


In [32]:
#test data preparation
test_X = []
test_Y = []
for l in test_lines:
  s = l.strip().split()
  words = [w.strip().split('/')[0] for w in s]
  labels = [w.strip().split('/')[1] for w in s]
  words = words[:maxlen]
  labels = labels[:maxlen]
  temp_x = []
  temp_y = []
  temp_x = [word2id[w] if w in word2id else 0 for w in words]
  temp_y = []
  for label in labels:
    y = [0]*len(class2id)
    y[class2id[label]] = 1
    temp_y.append(y)
  padlen = maxlen - len(words)
  temp_x += [0]*padlen
  temp_y += ([[0]*len(class2id)])*padlen
  test_X.append(temp_x)
  test_Y.append(temp_y)
print (len(test_X),len(test_Y))
print (test_X[0],test_Y[0])
test_X = np.array(test_X)
test_Y = np.array(test_Y)


20 20
[0, 0, 962, 0, 512, 0, 155, 0, 0, 0] [[0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 1]]


In [33]:
##Define a sequence tagging model
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
input = Input(shape=(maxlen,))
model = Embedding(input_dim=len(word2id), output_dim=50, input_length=maxlen)(input)  # 50-dim embedding
model = Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.1))(model)
out = TimeDistributed(Dense(len(class2id), activation="softmax"))(model)
model = Model(input, out)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(train_X, train_Y, batch_size=32, epochs=5)


Epoch 1/5
3/3 [==============================] - 0s 26ms/step - loss: 1.5410 - accuracy: 0.3050
Epoch 2/5
3/3 [==============================] - 0s 23ms/step - loss: 1.4640 - accuracy: 0.4288
Epoch 3/5
3/3 [==============================] - 0s 21ms/step - loss: 1.2802 - accuracy: 0.4288
Epoch 4/5
3/3 [==============================] - 0s 25ms/step - loss: 1.1724 - accuracy: 0.4288
Epoch 5/5
3/3 [==============================] - 0s 24ms/step - loss: 1.1138 - accuracy: 0.4288


In [34]:
res = model.evaluate(test_X,test_Y)

1/1 [==============================] - 0s 2ms/step - loss: 1.1073 - accuracy: 0.3150


In [36]:
p = model.predict(test_X)

In [37]:
p[0]

array([[0.05224843, 0.04564307, 0.09172364, 0.02241478, 0.21149153,
        0.5764786 ],
       [0.07095962, 0.06443349, 0.11344644, 0.03761691, 0.22308521,
        0.4904584 ],
       [0.08682059, 0.07997806, 0.12735948, 0.05392319, 0.22591677,
        0.4260019 ],
       [0.09410006, 0.08800966, 0.13712402, 0.06592254, 0.22675034,
        0.3880934 ],
       [0.09692826, 0.08819847, 0.13608973, 0.073116  , 0.2292716 ,
        0.3763959 ],
       [0.09365502, 0.0845281 , 0.13592531, 0.07558797, 0.23787919,
        0.37242436],
       [0.08287013, 0.07038967, 0.12144737, 0.06833199, 0.2510229 ,
        0.4059379 ],
       [0.06810728, 0.05413559, 0.10565104, 0.05660209, 0.2626846 ,
        0.4528194 ],
       [0.0460678 , 0.03254114, 0.07706073, 0.03711433, 0.27287576,
        0.53434026],
       [0.02316098, 0.01379549, 0.04389734, 0.01763576, 0.2630547 ,
        0.6384558 ]], dtype=float32)